# **Extract Transform Load (ETL) Lab**

In [123]:
import glob                         # this module helps in selecting files 
import zipfile                      # this module helps in unziping files
import pandas as pd                 # this module helps in processing CSV,JSON files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime


### Unzip Files

In [124]:
with zipfile.ZipFile("datasource.zip", 'r') as zip_ref:
    zip_ref.extractall("dealership_data")


## Extract

CSV format

In [125]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

JSON format

In [126]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

XML format

In [127]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    car_model_list = []
    year_of_manufacture_list = []
    price_list = []
    fuel_list = []

    for person in root:
        car_model = person.find("car_model").text
        car_model_list.append(car_model)
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        year_of_manufacture_list.append(year_of_manufacture)
        price = float(person.find("price").text)
        price_list.append(price)
        fuel = person.find("fuel").text
        fuel_list.append(fuel)
    df = pd.DataFrame({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel},index=[0])
    dataframe = pd.concat([dataframe,df])
    return dataframe

Extract Function

In [128]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = pd.concat([extracted_data,extract_from_csv(csvfile)])
        
    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = pd.concat([extracted_data,extract_from_json(jsonfile)])
    
    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = pd.concat([extracted_data,extract_from_xml(xmlfile)])
        
    return extracted_data

## Transform

In [129]:
def transform(data):
    data['price'] = round(data.price,2)
    return data


## Load

In [130]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

## Logging

In [131]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## ETL Process

In [132]:
targetfile = "dealership_transformed_data.csv"

In [133]:
log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")